# 0 Intro

This notebook is a quick intro to [GeoPandas](https://geopandas.org/en/stable/index.html). You may also want to go through the official [intro notebook](https://geopandas.org/en/stable/getting_started/introduction.html) at GeoPandas document site.

> GeoPandas is an open source project to make working with geospatial data in python easier. GeoPandas extends the datatypes used by [pandas](http://pandas.pydata.org/) to allow spatial operations on geometric types. Geometric operations are performed by [shapely](https://shapely.readthedocs.io/en/stable/). Geopandas further depends on [fiona](https://fiona.readthedocs.io/en/stable/) for file access and [matplotlib](https://matplotlib.org/) for plotting.
>
> -- from [GeoPandas Doc](https://geopandas.org/en/stable/index.html)

# 1 Setup

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd

In [ ]:
print(f"matplotlib version: {mpl.__version__}")
print(f"geopandas version: {gpd.__version__}")
print(f"pandas version: {pd.__version__}")

In [ ]:
# display all table columns
pd.set_option('display.max_columns', None)

In [ ]:
# get the Canada province boundary shape files (the digital boundary files) from Stats Canada
# the digital boundary files portray the full extent of the geographic areas, including the coastal water area
# https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/boundary-limites/index2021-eng.cfm?year=21
!wget --quiet https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/boundary-limites/files-fichiers/lpr_000a21a_e.zip

In [ ]:
# check what's in the zipped file (not unzipping it)
import zipfile

with zipfile.ZipFile("lpr_000a21a_e.zip", mode="r") as archive:
    archive.printdir()

# 2 Read data files

In [ ]:
# read the zipped shape files, and store the data into a geo-dataframe
shp_zip = "zip://lpr_000a21a_e.zip"
canada = gpd.read_file(shp_zip)

In [ ]:
# check columns and data types
canada.dtypes

In [ ]:
# check the first few rows
canada.head(5)

# 3 Coordinate Reference System (CRS)

> The CRS is a framework used to precisely measure locations on the surface of Earth as coordinates.
>
> -- from [wiki](https://en.wikipedia.org/wiki/Spatial_reference_system).

Any geospatial dataset comes with a CRS. Without it, you cannot plot or manipulate the data correctly.

There are two main types of CRSs.

1. Geographic Coordinate Systems (GCS) measures locations directly on the Earth using latitude and longitude.

2. Projected Coordinate Systems (PCS) models the Earth as a 2D plane and measures locations using a standardized cartesian coordinate system. PCS therefore involves a map projection from the Earth's 3D surface to a 2D plane.

Because there are many ways/specifications to model the Earth (as an ellipsoid), and many ways/specifications to project the Earth to a 2D plane, there are many CRSs. A CRS specification essentially consists of a "stack" of dependent specifications depending on the Earth model and projection.

The details of CRS are quite complex, I encourage you to read an excellent tutorial on CRS [here](https://docs.qgis.org/3.34/en/docs/gentle_gis_introduction/coordinate_reference_systems.html). After that, refer to the [CRS wikipedia article](https://en.wikipedia.org/wiki/Spatial_reference_system) for further details.

You can use Geopandas to check the CRS of your data and convert between CRSs easily. For example, the CRS of our data (i.e., the geo-spatial column `geometry`) is called NAD83 / Statistics Canada Lambert. It is a CRS specifically designed for use in Canada. You can find this information by displaying the data object/`GeoDataFrame`'s `crs` attribute. You can convert the data to other CRS by calling the `GeoDataFrame`'s `to_crs()` method.

In [ ]:
# Check the Coordinate Reference System (CRS) of the data
# https://docs.qgis.org/3.34/en/docs/gentle_gis_introduction/coordinate_reference_systems.html
canada.crs

In [ ]:
# produce a plot in the current CRS
canada.plot()

In [ ]:
# convert to another CRS
canada.to_crs(epsg=4326).crs

In [ ]:
# convert to another CRS and show the first few rows
# note the changes in the geometry column
canada.to_crs(epsg=4326).head()

In [ ]:
# convert to another CRS and plot
canada.to_crs(epsg=4326).plot()

In [ ]:
# plot in three different CRSs
# https://www.tomasbeuzen.com/python-for-geospatial-analysis/chapters/chapter1_intro-to-spatial.html
fig, axs = plt.subplots(1, 3, figsize=(10, 8))
crs_list = [("WGS 84", "EPSG:4326"), ("Stats Canada Lambert", "EPSG:3347"), ("UTM 10N", "EPSG:32610")]
for n, (name, epsg) in enumerate(crs_list):
    # convert to a different Coordinate Reference System (CRS)
    canada.to_crs(epsg).plot(ax=axs[n], edgecolor='white', linewidth=1)
    axs[n].set_title(name)
plt.tight_layout()

# 4 Geospatial operations

Geopandas makes geospatial operations easy. Below is an example of [calculating areas](https://geopandas.org/en/stable/getting_started/introduction.html#Measuring-area) bounded by polygons. Other geospatial operations Geopandas can perform include 1) [distance calculation between points](https://geopandas.org/en/stable/getting_started/introduction.html#Measuring-distance); 2) [spatial join](https://geopandas.org/en/stable/docs/user_guide/mergingdata.html); 3) [spatial aggregation](https://geopandas.org/en/stable/docs/user_guide/aggregation_with_dissolve.html); and many more.

In [ ]:
# calculate area bounded by the "geometry"/polygon (km^2)
# "For operations that rely on distance or area, you always need to use a projected
# CRS (in meters, feet, kilometers etc.) not a geographic one (in degrees)."
# use GeoSeries.to_crs() to project geometries to a planar CRS before using this function
canada["area"] = canada.area / 1000000

# note the new "area" covers off-shore area too
canada[["PREABBR", "LANDAREA", "area"]]

# 5 Plot maps

GeoPandas can also plot maps (as we have already seen above). You can also pass a `column` argument to the `plot()` method to color code a map.

In [ ]:
# plot a choropleth map based on each province's land area
canada.plot(column="LANDAREA", legend=True)

Building upon other open source projects such as [folium/leaflet.js](https://python-visualization.github.io/folium/latest/) and [mapclassify](https://github.com/pysal/mapclassify), Geopandas makes creating interactive maps an easy task too.

In [ ]:
# install mapclassify for an interactive map
!pip install --quiet mapclassify

In [ ]:
# plot an interative choropleth map based on each province's land area
canada.explore(column="area", legend=False, width="50%", height="50%")